# 📈 Quant Apprentice: Final Submission

**Version 2.0: Agentic Graph Workflow**

This notebook represents the final, self-contained version of the Quant Apprentice project. It implements a sophisticated, multi-step financial analysis agent using a stateful graph architecture.

### Core Agentic Capabilities Demonstrated:
- **Planning & Execution**: The agent follows a multi-node graph that defines a clear, repeatable research plan (see Section 4).
- **Dynamic Tool Use**: It leverages external APIs for real-time financial, news, and SEC filing data (see Section 3.1).
- **Self-Reflection**: It uses an explicit Evaluator-Optimizer loop within the graph to critique and refine its own analysis (see Section 4, `evaluate_report_node` and `should_refine_or_end`).
- **Learning (Memory)**: It incorporates a vector database (ChromaDB) to perform semantic searches on past analyses, demonstrating a Retrieval-Augmented Generation (RAG) pattern (see Section 3.2 and graph nodes `retrieve_from_memory_node`, `save_to_memory_node`).

### Agentic Workflow Patterns Implemented:
1.  **Prompt Chaining**: The `analyze_article_chain` function processes raw news into structured JSON (see Section 3.3).
2.  **Task Routing**: The `route_and_execute_task` function directs data to specialized analyst prompts (see Section 4).
3.  **Evaluator-Optimizer Loop**: The graph's conditional edge (`should_refine_or_end`) and refinement loop embody this pattern (see Section 4).

This notebook contains all necessary code to run the agent. Simply install the dependencies, add your API keys, and execute the cells in order.

## 1. Environment Setup

In [ ]:
# This cell installs all necessary libraries.
%pip install --upgrade --quiet google-generativeai langgraph yfinance fredapi newsapi-python sec-api python-dotenv pandas chromadb sentence-transformers

In [1]:
# Import all required libraries for the agent
import os
import json
import re
from datetime import datetime
from typing import TypedDict, List

# External Data and AI Libraries
import yfinance as yf
from fredapi import Fred
from newsapi import NewsApiClient
from sec_api import QueryApi
import google.generativeai as genai
import chromadb
from dotenv import load_dotenv

# Agentic Graph Library
from langgraph.graph import StateGraph, END

# Notebook Display Utilities
from IPython.display import display, Markdown

print("✅ All libraries imported successfully.")

✅ All libraries imported successfully.


## 2. API Key Configuration

**Action Required**: You must add your API keys to run this notebook. The easiest way is to create a `.env` file in the same directory as this notebook and paste the following content into it, replacing `"YOUR_KEY_HERE"` with your actual keys.

In [ ]:
load_dotenv()
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
llm = genai.GenerativeModel('gemini-2.5-pro', generation_config={'temperature': 0.2})
fred = Fred(api_key=os.getenv("FRED_API_KEY"))
newsapi = NewsApiClient(api_key=os.getenv("NEWS_API_KEY"))
query_api = QueryApi(api_key=os.getenv("SEC_API_KEY"))

## 3. Self-Contained Agent Components

This section consolidates all the Python code from the project's external modules (`tools`, `workflows`, `memory`) into this notebook.

### 3.1. Tool Belt Functions

In [ ]:
def get_stock_fundamentals(ticker_symbol: str) -> dict:
    """Fetches key fundamental data for a given stock ticker using yfinance."""
    print(f"--- [Tool Action]: Fetching fundamental data for {ticker_symbol}... ---")
    try:
        stock = yf.Ticker(ticker_symbol)
        info = stock.info
        fundamentals = {
            "ticker": ticker_symbol,
            "companyName": info.get("longName"),
            "sector": info.get("sector"),
            "industry": info.get("industry"),
            "marketCap": info.get("marketCap"),
            "enterpriseValue": info.get("enterpriseValue"),
            "trailingPE": info.get("trailingPE"),
            "forwardPE": info.get("forwardPE"),
            "trailingEps": info.get("trailingEps"),
            "priceToBook": info.get("priceToBook"),
            "dividendYield": info.get("dividendYield"),
            "payoutRatio": info.get("payoutRatio"),
        }
        print(f"--- [Tool Success]: Successfully fetched fundamentals for {ticker_symbol}. ---")
        return fundamentals
    except Exception as e:
        error_message = f"Could not fetch data for {ticker_symbol}. Ticker might be invalid. Details: {e}"
        print(f"--- [Tool Error]: {error_message} ---")
        return {"error": error_message}

def get_macro_economic_data(api_key: str) -> dict:
    """Fetches key US macroeconomic indicators from the FRED API."""
    print("--- [Tool Action]: Fetching macroeconomic data from FRED... ---")
    try:
        fred = Fred(api_key=api_key)
        series_ids = {
            "GDP_Growth": "GDP",
            "UnemploymentRate": "UNRATE",
            "InflationRate_CPI": "CPIAUCSL",
            "EffectiveFedFundsRate": "FEDFUNDS",
        }
        macro_data = {}
        for name, series_id in series_ids.items():
            data = fred.get_series_latest_release(series_id)
            macro_data[name] = data.iloc[-1]
        print("--- [Tool Success]: Successfully fetched macroeconomic data. ---")
        return macro_data
    except Exception as e:
        error_message = f"Could not fetch FRED data. Check API key or connection. Details: {e}"
        print(f"--- [Tool Error]: {error_message} ---")
        return {"error": error_message}

def get_company_news(company_name: str, api_key: str, num_articles: int = 3) -> dict:
    """Fetches and processes top news headlines for a given company using the NewsAPI."""
    print(f"--- [Tool Action]: Fetching top {num_articles} news articles for {company_name}... ---")
    try:
        newsapi = NewsApiClient(api_key=api_key)
        top_headlines = newsapi.get_everything(
            q=company_name,
            language='en',
            sort_by='relevancy',
            page_size=num_articles
        )
        if top_headlines['status'] != 'ok':
            return {"error": "Failed to fetch news from NewsAPI."}
        processed_articles = []
        for article in top_headlines['articles']:
            processed_articles.append({
                "source": article['source']['name'],
                "title": article['title'],
                "url": article['url'],
                "publishedAt": article['publishedAt'],
                "content": article.get('content', 'No content available.')
            })
        print(f"--- [Tool Success]: Successfully fetched {len(processed_articles)} articles. ---")
        return {"articles": processed_articles}
    except Exception as e:
        error_message = f"An error occurred while fetching news: {e}"
        print(f"--- [Tool Error]: {error_message} ---")
        return {"error": error_message}

def get_latest_sec_filings(company_ticker: str, api_key: str) -> dict:
    """Fetches the most recent 10-K and 10-Q filings for a company."""
    print(f"--- [Tool Action]: Fetching latest SEC filings for {company_ticker}... ---")
    try:
        queryApi = QueryApi(api_key=api_key)
        query = {
          "query": { "query_string": {
              "query": f"ticker:{company_ticker} AND formType:\"10-K\" OR formType:\"10-Q\""
          }},
          "from": "0",
          "size": "1",
          "sort": [{ "filedAt": { "order": "desc" } }]
        }
        response = queryApi.get_filings(query)
        if not response['filings']:
            return {"error": f"No recent 10-K or 10-Q found for {company_ticker}."}
        latest_filing = response['filings'][0]
        filing_url = latest_filing['linkToFilingDetails']
        print(f"--- [Tool Success]: Found latest filing: {latest_filing['formType']} filed on {latest_filing['filedAt'][:10]} ---")
        # In a real-world scenario, you would use an extraction API to get the text.
        # For this project, we simulate the output to demonstrate the capability.
        return {
            "filing_type": latest_filing['formType'],
            "filed_at": latest_filing['filedAt'],
            "link_to_filing": filing_url,
            "summary_of_risk_factors": f"Extracted key risk factors related to competition and market trends for {company_ticker}.",
            "summary_of_mdna": f"Extracted management's discussion on financial performance and future outlook for {company_ticker}."
        }
    except Exception as e:
        print(f"--- [Tool Error]: Failed to fetch SEC filings. Details: {e} ---")
        return {"error": str(e)}

### 3.2. Vector Memory System

In [ ]:
class VectorMemory:
    """A class to manage agent memory using a ChromaDB vector database."""
    def __init__(self, db_path: str = "chroma_db_memory"):
        print(f"--- [Memory]: Initializing ChromaDB at {db_path} ---")
        self.client = chromadb.PersistentClient(path=db_path)
        self.collection = self.client.get_or_create_collection(name="quant_apprentice_memory")

    def add_analysis(self, ticker: str, report_text: str):
        print(f"--- [Memory]: Adding analysis for {ticker} to vector memory... ---")
        try:
            current_date = datetime.now().strftime("%Y-%m-%d-%H:%M:%S")
            unique_id = f"{ticker}_{current_date}"
            self.collection.add(
                documents=[report_text],
                metadatas=[{"ticker": ticker, "date": current_date}],
                ids=[unique_id]
            )
            print(f"--- [Memory]: Successfully added document with ID: {unique_id} ---")
        except Exception as e:
            print(f"--- [Memory Error]: Failed to add analysis for {ticker}. Details: {e} ---")

    def query_memory(self, query_text: str, n_results: int = 1) -> list:
        print(f"--- [Memory]: Querying memory with: '{query_text}' ---")
        try:
            results = self.collection.query(query_texts=[query_text], n_results=n_results)
            return results.get('documents', [[]])[0]
        except Exception as e:
            print(f"--- [Memory Error]: Failed to query memory. Details: {e} ---")
            return []

### 3.3. Workflow Function (Prompt Chaining)

In [ ]:
def analyze_article_chain(article_content: str, llm_model: genai.GenerativeModel) -> dict:
    """(WORKFLOW 1: PROMPT CHAINING) Analyzes a news article using a structured prompt for financial sentiment."""
    print("--- [Workflow Action]: Starting Refined News Analysis Chain... ---")
    prompt = f"""
    You are a skeptical financial analyst. Analyze the following news article from a cautious investor's perspective.
    **Analysis Steps:**
    1. **Reasoning:** In one sentence, explain the financial impact on the company's bottom line or market position.
    2. **Sentiment Classification:** Based only on your reasoning, classify the sentiment as 'Positive', 'Negative', or 'Neutral' according to the rubric.
    3. **Key Takeaways:** Extract the 3 most important bullet-point takeaways.
    4. **Summary:** Provide a concise 2-sentence summary.
    **Sentiment Rubric:**
    - **Positive**: Favorable impact on revenue, earnings, or market share.
    - **Negative**: Direct risk to earnings, operations, or reputation.
    - **Neutral**: Informational but no clear, immediate financial impact.
    **Article Content:**
    ---
    {article_content}
    ---
    Provide the output in a single, valid JSON object with keys: "reasoning", "sentiment", "key_takeaways", "summary".
    """
    try:
        if not article_content or len(article_content.strip()) < 20:
            return {"error": "Content too short for analysis."}
        response = llm_model.generate_content(prompt)
        cleaned_response = re.sub(r"```json\n?|```", "", response.text)
        analysis_result = json.loads(cleaned_response)
        print("--- [Workflow Success]: Refined News Analysis completed. ---")
        return analysis_result
    except Exception as e:
        error_message = f"Failed to analyze article. Details: {e}"
        print(f"--- [Workflow Error]: {error_message} ---")
        return {"error": error_message}

## 4. Agent Graph Definition

This is the core of the Version 2 agent. It defines the state, nodes, and edges of the agentic workflow using LangGraph. All prompts and workflow logic are consolidated here.

In [ ]:
# --- PROMPT TEMPLATES --- 
FINANCIAL_ANALYST_PROMPT = """
As a Quantitative Financial Analyst, analyze the provided key financial metrics. Focus on valuation, profitability, and financial health. Provide a 3-5 bullet point summary highlighting strengths and weaknesses. Be objective and data-driven.
**Financial Data:**
{financial_data}
"""

NEWS_ANALYST_PROMPT = """
As an Investment News Analyst, interpret the provided structured news analyses. What is the likely short-term impact on the company's stock price? Provide a 2-3 sentence summary of your impact assessment.
**Structured News Analysis:**
{news_analysis}
"""

MARKET_ANALYST_PROMPT = """
As a Macroeconomic Analyst, provide market context. How might the current economic environment (inflation, interest rates, GDP) affect the broader stock market and the company's sector? Provide a 2-3 sentence summary.
**Macroeconomic Data:**
{macro_data}
"""

SYNTHESIS_PROMPT_TEMPLATE = """
You are a Chief Investment Strategist. Synthesize the reports from your specialist teams, your memory, and SEC filings into a final investment report for {company_name}.
The report must include: Executive Summary, Key Findings, Final Recommendation (Buy/Hold/Sell), and Justification.

**PRIOR ANALYSIS (from memory):**
{past_analysis}
---
**LATEST SEC FILING INSIGHTS:**
{sec_filings_summary}
---
**CURRENT SPECIALIST REPORTS:**
1. Quantitative Financial Analysis:
{financial_analysis}
2. News Impact Analysis:
{news_impact_analysis}
3. Macroeconomic Context:
{market_context_analysis}
"""

EVALUATOR_PROMPT_TEMPLATE = """
You are a skeptical Risk Manager. Critique the following draft report. Identify potential weaknesses, biases, or gaps. Focus on whether the recommendation is well-supported. Provide feedback in a concise, 2-4 bullet point list.
**DRAFT REPORT TO EVALUATE:**
{draft_report}
"""

REFINEMENT_PROMPT_TEMPLATE = """
You are the Chief Investment Strategist. Revise your report based on the Risk Manager's feedback to create a more robust final version.
**Original Specialist Reports & Data:**
Financial Analysis: {financial_analysis}
News Analysis: {news_impact_analysis}
Macro Context: {market_context_analysis}

**Risk Manager's Feedback:**
{feedback}

Now, generate the final, refined investment report for {company_name}.
"""

# --- AGENT STATE --- 
class AgentState(TypedDict):
    company_name: str
    company_ticker: str
    financial_data: dict
    macro_data: dict
    news_data: dict
    sec_filings_data: dict
    past_analysis: str
    structured_news_analysis: dict
    financial_analysis: str
    news_impact_analysis: str
    market_context_analysis: str
    draft_report: str
    feedback: str
    final_report: str
    revision_count: int

# --- WORKFLOW & NODE FUNCTIONS ---

def route_and_execute_task(task_type: str, data: dict, llm_model: genai.GenerativeModel) -> str:
    """(WORKFLOW 2: TASK ROUTING) Routes data to the correct specialist analyst prompt."""
    prompt_map = {
        'analyze_financials': FINANCIAL_ANALYST_PROMPT.format(financial_data=data),
        'analyze_news_impact': NEWS_ANALYST_PROMPT.format(news_analysis=data),
        'analyze_market_context': MARKET_ANALYST_PROMPT.format(macro_data=data)
    }
    prompt = prompt_map.get(task_type)
    if not prompt:
        return f"--- [Router Error]: Invalid task type: {task_type} ---"
    print(f"--- [Router]: Routing to {task_type.split('_')[1].capitalize()} Analyst... ---")
    try:
        response = llm_model.generate_content(prompt)
        print(f"--- [Router]: Specialist analysis complete. ---")
        return response.text
    except Exception as e:
        return f"--- [Router Error]: {e} ---"

def gather_data_node(state: AgentState):
    print("--- [Node]: Gathering Data... ---")
    company_name = state['company_name']
    company_ticker = state['company_ticker']
    financial_data = get_stock_fundamentals(company_ticker)
    macro_data = get_macro_economic_data(os.getenv("FRED_API_KEY"))
    news_data = get_company_news(company_name, os.getenv("NEWS_API_KEY"), num_articles=3)
    return {"financial_data": financial_data, "macro_data": macro_data, "news_data": news_data}

def retrieve_from_memory_node(state: AgentState):
    print("--- [Node]: Retrieving from Vector Memory... ---")
    company_name = state['company_name']
    memory = VectorMemory()
    query = f"What was my past analysis and conclusion for {company_name}?"
    results = memory.query_memory(query, n_results=1)
    past_analysis = "\n".join(results) if results else "No prior analysis found in memory."
    print(f"--- [Memory]: Found relevant past analysis." if results else "--- [Memory]: No relevant past analysis found. ---")
    return {"past_analysis": past_analysis}

def sec_filings_node(state: AgentState):
    print("--- [Node]: Fetching SEC Filings... ---")
    company_ticker = state['company_ticker']
    sec_data = get_latest_sec_filings(company_ticker, os.getenv("SEC_API_KEY"))
    return {"sec_filings_data": sec_data}

def specialist_analysis_node(state: AgentState):
    print("--- [Node]: Performing Specialist Analysis... ---")
    processed_analyses = [analyze_article_chain(article['content'], llm) for article in state["news_data"].get("articles", [])]
    structured_news_analysis = {"news_items": processed_analyses}
    financial_analysis = route_and_execute_task('analyze_financials', state['financial_data'], llm)
    news_impact_analysis = route_and_execute_task('analyze_news_impact', structured_news_analysis, llm)
    market_context_analysis = route_and_execute_task('analyze_market_context', state['macro_data'], llm)
    return {"structured_news_analysis": structured_news_analysis, "financial_analysis": financial_analysis, "news_impact_analysis": news_impact_analysis, "market_context_analysis": market_context_analysis}

def synthesize_report_node(state: AgentState):
    print("--- [Node]: Synthesizing Draft Report... ---")
    prompt = SYNTHESIS_PROMPT_TEMPLATE.format(
        company_name=state['company_name'],
        past_analysis=state['past_analysis'],
        sec_filings_summary=state.get('sec_filings_data', 'Not available'),
        financial_analysis=state['financial_analysis'],
        news_impact_analysis=state['news_impact_analysis'],
        market_context_analysis=state['market_context_analysis']
    )
    draft_report = llm.generate_content(prompt).text
    revision_count = state.get('revision_count', 0) + 1
    return {"draft_report": draft_report, "revision_count": revision_count}

def evaluate_report_node(state: AgentState):
    print("--- [Node]: Evaluating Draft Report... ---")
    prompt = EVALUATOR_PROMPT_TEMPLATE.format(draft_report=state['draft_report'])
    feedback = llm.generate_content(prompt).text
    return {"feedback": feedback}

def refine_report_node(state: AgentState):
    print("--- [Node]: Refining Final Report... ---")
    prompt = REFINEMENT_PROMPT_TEMPLATE.format(
        company_name=state['company_name'],
        financial_analysis=state['financial_analysis'],
        news_impact_analysis=state['news_impact_analysis'],
        market_context_analysis=state['market_context_analysis'],
        feedback=state['feedback']
    )
    final_report = llm.generate_content(prompt).text
    return {"final_report": final_report}

def save_to_memory_node(state: AgentState):
    print("--- [Node]: Saving to Vector Memory... ---")
    company_ticker = state['company_ticker']
    report_to_save = state.get('final_report') or state.get('draft_report')
    if report_to_save:
        memory = VectorMemory()
        memory.add_analysis(company_ticker, report_to_save)
    return {}

# --- CONDITIONAL EDGE --- 
def should_refine_or_end(state: AgentState):
    """(WORKFLOW 3: EVALUATOR-OPTIMIZER) Uses the LLM to decide whether to refine the report or end the process."""
    print("--- [Conditional Edge]: Using LLM to check feedback... ---")
    if state['revision_count'] > 1:
        print("--- [Decision]: Maximum revisions reached. Ending. ---")
        return "end"
    decision_prompt = f"""You are a gatekeeper. Decide if a report needs revision based on the following feedback. If the feedback points out flaws or weaknesses, a revision is required. Answer ONLY with 'Yes' or 'No'.\nFeedback:\n{state['feedback']}"""
    try:
        response = llm.generate_content(decision_prompt)
        decision = response.text.strip().lower()
        if "yes" in decision:
            print("--- [LLM Decision]: Feedback requires revision. Refining report. ---")
            return "refine"
        else:
            print("--- [LLM Decision]: Feedback is positive. Ending. ---")
            return "end"
    except Exception as e:
        print(f"--- [Error]: Could not make a decision. Defaulting to end. Details: {e} ---")
        return "end"

# --- GRAPH ASSEMBLY --- 
workflow = StateGraph(AgentState)
workflow.add_node("gather_data", gather_data_node)
workflow.add_node("retrieve_from_memory", retrieve_from_memory_node)
workflow.add_node("fetch_sec_filings", sec_filings_node)
workflow.add_node("analyze_specialists", specialist_analysis_node)
workflow.add_node("synthesize_report", synthesize_report_node)
workflow.add_node("evaluate_report", evaluate_report_node)
workflow.add_node("refine_report", refine_report_node)
workflow.add_node("save_to_memory", save_to_memory_node)

workflow.set_entry_point("gather_data")
workflow.add_edge("gather_data", "retrieve_from_memory")
workflow.add_edge("retrieve_from_memory", "fetch_sec_filings")
workflow.add_edge("fetch_sec_filings", "analyze_specialists")
workflow.add_edge("analyze_specialists", "synthesize_report")
workflow.add_edge("synthesize_report", "evaluate_report")
workflow.add_edge("refine_report", "save_to_memory")
workflow.add_edge("save_to_memory", END)

workflow.add_conditional_edges(
    "evaluate_report",
    should_refine_or_end,
    {"refine": "refine_report", "end": "save_to_memory"}
)

app = workflow.compile()
print("✅ Agent Graph compiled successfully.")

## 5. Agent Execution

In [ ]:
# --- Agent Configuration ---
company_name = "NVIDIA"
company_ticker = "NVDA"

initial_state = {
    "company_name": company_name,
    "company_ticker": company_ticker,
    "revision_count": 0
}

print(f"🚀 Starting Quant Apprentice agent for {company_name}...")

# --- Run the Agentic Graph ---
final_state = app.invoke(initial_state)

print("\n" + "="*50)
print("✅ Agent run complete.")
print("="*50 + "\n")

## 6. Display Final Report

In [ ]:
if final_state:
    display(Markdown(f"# Final Investment Report: {company_name} ({company_ticker})"))
    
    # The final report will either be in 'final_report' (if refined) or 'draft_report' (if the first draft was approved).
    report_to_display = final_state.get('final_report', final_state.get('draft_report', "*No report was generated.*"))
    feedback = final_state.get('feedback', "*No feedback was generated.*")
    
    display(Markdown("---"))
    display(Markdown("## Final Critic's Feedback:"))
    display(Markdown(feedback))
    
    display(Markdown("---"))
    display(Markdown("## **Final Report Delivered:**"))
    display(Markdown(report_to_display))
else:
    display(Markdown(f"# Agent Run Failed for {company_name}"))